In [31]:
import numpy as np
import pandas as pd
import os, itertools, csv
from bs4 import BeautifulSoup
import re
from pytrends.request import TrendReq
from datetime import *
import time
import nltk
from sklearn.model_selection import train_test_split

# load the dataset
df = pd.read_csv('./data/train.csv')
print(df.head())
print(df.shape)


   Id  Popularity                                       Page content
0   0          -1  <html><head><div class="article-info"> <span c...
1   1           1  <html><head><div class="article-info"><span cl...
2   2           1  <html><head><div class="article-info"><span cl...
3   3          -1  <html><head><div class="article-info"><span cl...
4   4          -1  <html><head><div class="article-info"><span cl...
(27643, 3)


In [68]:
X_train = df.loc[:19999, :]
print(X_train.shape)
X_valid = df.loc[20000:, :]
print(X_valid.shape)
X_train = X_train.reset_index(drop=True)
X_valid = X_valid.reset_index(drop=True)

(20000, 3)
(7643, 3)


In [22]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import nltk
from nltk.stem.porter import PorterStemmer

nltk.download('wordnet')
stop_words = set(stopwords.words("english"))

def get_category(text: str) -> list:
    res = list()
    soup = BeautifulSoup(text, "html.parser")
    soup = soup.find('footer',{'class':'article-topics'})
    cats = soup.find_all('a', {'href': re.compile('/category/*')})
    for cat in cats:
        res.append(cat.get_text().lower())
    porter = PorterStemmer()
    res = [porter.stem(word) for word in res if not word in  
        stop_words]
    return list(set(res))

res = []
for i in range(0, df.shape[0]):
    res.append(get_category(df['Page content'][i]))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yuwei\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
res

[['u.s.', 'space', 'world', 'challeng', 'earth', 'asteroid'],
 ['googl',
  'tech',
  'open sourc',
  'u.s.',
  'apps and softwar',
  'patent lawsuit theat',
  'opn pledg',
  'software pat'],
 ['sport', 'entertain', 'nfl draft', 'nfl', 'televis'],
 ['video', 'watercool', 'sport'],
 ['instagram video', 'sport', 'instagram', 'entertain', 'nfl'],
 ['lobbyist',
  'govern',
  'u.s.',
  'startup',
  'lobbi',
  'internet',
  'mobil',
  'polit',
  'internet service provid',
  'world'],
 ['music', 'iggy azalea', 'entertain', 'parodi', 'video', 'funni'],
 ['bill cosbi', 'televis', 'entertain'],
 ['vending machin', 'tech', 'apps and softwar', 'mobil', 'gadget'],
 ['busi', 'media', 'paywal', 'the new york tim'],
 ['astronomi', 'moon', 'space', 'world', 'saturn'],
 ['mad men', 'humor', 'photographi', 'televis', 'watercool'],
 ['u.s.', 'apps and softwar', 'tech', 'nevada'],
 ['busi', 'u.s.', 'appl', 'dev & design', 'steve job'],
 ['soccer', 'sport', 'world cup', "women's world cup", 'uswnt', 'enterta

In [38]:
pop = X_train[X_train.loc[:,"Popularity"] > 0]
unpop = X_train[X_train.loc[:,"Popularity"] < 0]
pop = pop.reset_index(drop=True)
unpop = unpop.reset_index(drop=True)
pop

,Id,Popularity,Page content
0,1,1,"<html><head><div class=""article-info""><span cl..."
1,2,1,"<html><head><div class=""article-info""><span cl..."
2,6,1,"<html><head><div class=""article-info""><span cl..."
3,8,1,"<html><head><div class=""article-info""><span cl..."
4,11,1,"<html><head><div class=""article-info""><span cl..."
...,...,...,...
9823,19984,1,"<html><head><div class=""article-info""><span cl..."
9824,19988,1,"<html><head><div class=""article-info""><span cl..."
9825,19993,1,"<html><head><div class=""article-info""><span cl..."
9826,19995,1,"<html><head><div class=""article-info""><span cl..."


In [48]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import nltk
from nltk.stem.porter import PorterStemmer

stop_words = set(stopwords.words("english"))

def get_category(text: str) -> list:
    res = list()
    soup = BeautifulSoup(text, "html.parser")
    soup = soup.find('footer',{'class':'article-topics'})
    cats = soup.find_all('a', {'href': re.compile('/category/*')})
    for cat in cats:
        res.append(cat.get_text().lower())
    porter = PorterStemmer()
    res = [porter.stem(word) for word in res if not word in  
        stop_words]
    return list(set(res))

pop_cat = []
for i in range(0, pop.shape[0]):
    pop_cat.extend(get_category(pop['Page content'][i]))

In [49]:
unpop_cat = []
for i in range(0, unpop.shape[0]):
    unpop_cat.extend(get_category(unpop['Page content'][i]))

In [50]:
pop_dict = {}

for cat in pop_cat:
    if cat in pop_dict:
        pop_dict[cat] = pop_dict[cat] + 1
    else:
        pop_dict[cat] = 1
pop_dict

{'googl': 383,
 'tech': 1997,
 'open sourc': 5,
 'u.s.': 1485,
 'apps and softwar': 1307,
 'patent lawsuit theat': 2,
 'opn pledg': 1,
 'software pat': 1,
 'sport': 524,
 'entertain': 1781,
 'nfl draft': 4,
 'nfl': 62,
 'televis': 862,
 'music': 489,
 'iggy azalea': 1,
 'parodi': 36,
 'video': 1347,
 'funni': 86,
 'vending machin': 6,
 'mobil': 1165,
 'gadget': 1004,
 'mad men': 16,
 'humor': 265,
 'photographi': 784,
 'watercool': 1822,
 'nevada': 1,
 'busi': 1549,
 'appl': 376,
 'dev & design': 586,
 'steve job': 20,
 'recal': 12,
 'food': 96,
 'viral video': 594,
 'wine': 4,
 'comcast': 14,
 'net neutrality ': 11,
 'transpar': 6,
 'transparency report': 6,
 'surveil': 51,
 'world': 2261,
 'ukraine ': 97,
 'alex trebek': 1,
 'mustach': 3,
 'jeopardi': 6,
 'convers': 378,
 'job': 260,
 'media': 359,
 'pricelin': 2,
 'job board': 7,
 'arduino': 3,
 'ada lovelac': 1,
 'empow': 1,
 'stem': 13,
 'raspberry pi': 4,
 'work & play': 539,
 'travel & leisur': 478,
 'educ': 52,
 'women in tech'

In [51]:
unpop_dict = {}

for cat in unpop_cat:
    if cat in unpop_dict:
        unpop_dict[cat] = unpop_dict[cat] + 1
    else:
        unpop_dict[cat] = 1
unpop_dict

{'u.s.': 1724,
 'space': 285,
 'world': 2565,
 'challeng': 4,
 'earth': 10,
 'asteroid': 18,
 'video': 1361,
 'watercool': 1840,
 'sport': 636,
 'instagram video': 7,
 'instagram': 138,
 'entertain': 1935,
 'nfl': 97,
 'lobbyist': 1,
 'govern': 20,
 'startup': 440,
 'lobbi': 2,
 'internet': 36,
 'mobil': 1166,
 'polit': 511,
 'internet service provid': 1,
 'bill cosbi': 5,
 'televis': 851,
 'busi': 1661,
 'media': 438,
 'paywal': 6,
 'the new york tim': 26,
 'astronomi': 20,
 'moon': 17,
 'saturn': 5,
 'soccer': 71,
 'world cup': 99,
 "women's world cup": 1,
 'uswnt': 1,
 'air safeti': 1,
 'travel & leisur': 418,
 'airplan': 11,
 'aviat': 9,
 'travel': 439,
 'googl': 354,
 'stock': 69,
 'larry pag': 4,
 'game of thron': 65,
 'ugc': 33,
 'easter': 7,
 'photographi': 724,
 'chris hadfield': 10,
 'gadget': 969,
 'amc': 10,
 'mad men': 7,
 'humor': 220,
 'nowthisnew': 2,
 'apps and softwar': 1216,
 'facebook': 362,
 'photo shar': 3,
 'social media': 570,
 'nobel': 2,
 'malala yousafzai': 4

In [58]:
pop_dict_1 = pop_dict
unpop_dict_1 = unpop_dict

In [59]:
coeff = {}

for key in pop_dict_1.keys():
    pop_num = pop_dict_1[key]
    unpop_num = 0
    if key in unpop_dict_1:
        unpop_num = unpop_dict_1.pop(key)
    coeff[key] = (pop_num-unpop_num) / (pop_num+unpop_num)
    
for key in unpop_dict_1.keys():
    coeff[key] = -1

In [69]:
X_valid.head()

,Id,Popularity,Page content
0,20000,-1,"<html><head><div class=""article-info""><span cl..."
1,20001,-1,"<html><head><div class=""article-info""><span cl..."
2,20002,1,"<html><head><div class=""article-info""><span cl..."
3,20003,-1,"<html><head><div class=""article-info""><span cl..."
4,20004,-1,"<html><head><div class=""article-info""><span cl..."


In [80]:
def get_category(text: str) -> list:
    res = list()
    soup = BeautifulSoup(text, "html.parser")
    soup = soup.find('footer',{'class':'article-topics'})
    cats = soup.find_all('a', {'href': re.compile('/category/*')})
    for cat in cats:
        res.append(cat.get_text().lower())
    porter = PorterStemmer()
    res = [porter.stem(word) for word in res if not word in  
        stop_words]
    return list(set(res))

res = []
for i in range(0, df.shape[0]):
    res.append(get_category(df['Page content'][i]))

In [81]:
cat_list = res
weight_list = []
for cats in cat_list:
    weight = 0
    for cat in cats:
        if cat in coeff:
            weight = weight + coeff[cat]
    weight_list.append(weight)
    
weight_list

[-0.707605506405546,
 0.46159014227103773,
 0.24830156128246453,
 -0.10663693800351691,
 -0.14101465833839147,
 -2.351532946825273,
 0.7441807615600041,
 -0.035020928416082425,
 0.5533786554019268,
 -0.6740126721310835,
 0.03703569429673287,
 0.5253817873629404,
 -0.038409857728962105,
 -0.22942392362894892,
 -2.1955172573720154,
 -1.2219945427404282,
 0.2984423676012461,
 0.29020327289024944,
 -0.36122084812304783,
 -1.455525886367008,
 -0.31356996295492073,
 -0.037071509414031846,
 0.030460411623437364,
 -0.031077232367230694,
 2.1651751271380015,
 0.7727333596149483,
 0.20664268214040132,
 -0.12298532979785817,
 1.0467358165213936,
 -2.3963254593175853,
 0.152793574013202,
 0.800697934276309,
 -0.3605388846649029,
 -0.16418222614121925,
 -0.6350642920310652,
 -2.0076826861961936,
 1.7307734737630873,
 0.1637644113778738,
 1.7976139346218085,
 0.5770814206694697,
 0.07710049199293462,
 -2.108105728128509,
 -0.3995325968712258,
 0.43172940659100967,
 -0.26552758502614204,
 -0.12928317

In [85]:
df2 = pd.read_csv('./data/test.csv')
print(df2.head())
print(df2.shape)

      Id                                       Page content
0  27643  <html><head><div class="article-info"><span cl...
1  27644  <html><head><div class="article-info"><span cl...
2  27645  <html><head><div class="article-info"><span cl...
3  27646  <html><head><div class="article-info"><span cl...
4  27647  <html><head><div class="article-info"><span cl...
(11847, 2)


In [86]:
def get_category(text: str) -> list:
    res = list()
    soup = BeautifulSoup(text, "html.parser")
    soup = soup.find('footer',{'class':'article-topics'})
    cats = soup.find_all('a', {'href': re.compile('/category/*')})
    for cat in cats:
        res.append(cat.get_text().lower())
    porter = PorterStemmer()
    res = [porter.stem(word) for word in res if not word in  
        stop_words]
    return list(set(res))

res2 = []
for i in range(0, df2.shape[0]):
    res2.append(get_category(df2['Page content'][i]))

In [87]:
cat_list = res2
weight_list = []
for cats in cat_list:
    weight = 0
    for cat in cats:
        if cat in coeff:
            weight = weight + coeff[cat]
    weight_list.append(weight)
    
weight_list

[-0.5148198985662944,
 -0.7913615158693416,
 0.12900990351852926,
 -1.4076119760577035,
 -0.6905285678928114,
 0.22813696632075967,
 -0.5418002468202858,
 0.9474087943920174,
 0.33419525333824496,
 -2.125948299754843,
 -0.17966186678825258,
 0,
 -0.07024288518153494,
 -0.07220684063810531,
 -2.1181396562580677,
 -0.6131027805630799,
 -1.0348909657320873,
 -0.055502943408606634,
 0.20203038000149526,
 0.9752134118370472,
 0.14947085189353942,
 -0.19112263531842252,
 0.21788635462832848,
 -1.4278123912346132,
 -0.005920720409127014,
 -0.13521718478827277,
 0.039035531860591756,
 0.43564635905455557,
 -0.4625824635065253,
 -0.035020928416082425,
 0.8169984313642695,
 0.004979071583917576,
 0.7845204830271628,
 -1.7396032331930127,
 0.517099825919973,
 -0.2487064116985377,
 -1.4836698495392195,
 0.004863007524378654,
 -0.5733026462390829,
 0.40615554217215283,
 0.27668847134795455,
 -0.35811992149628624,
 1.564185282577415,
 -0.9085576685102474,
 0.17626966022634974,
 -0.03489096573208723,

In [88]:
df2["cat_weight"] = weight_list

In [92]:
df2.to_csv("./data/test_cat_weight.csv")
df2.head()

,Id,Page content,cat_weight
0,27643,"<html><head><div class=""article-info""><span cl...",-0.514820
1,27644,"<html><head><div class=""article-info""><span cl...",-0.791362
2,27645,"<html><head><div class=""article-info""><span cl...",0.129010
3,27646,"<html><head><div class=""article-info""><span cl...",-1.407612
4,27647,"<html><head><div class=""article-info""><span cl...",-0.690529


In [82]:
df["cat_weight"] = weight_list

In [91]:
df.to_csv("./data/train_cat_weight.csv")

In [71]:
cat_list = res
weight_list = []
for cats in cat_list:
    weight = 0
    for cat in cats:
        if cat in coeff:
            weight = weight + coeff[cat]
    weight_list.append(weight)
    
weight_list

[-0.8387497017418277,
 -1.0902592385237577,
 0.18855292988911612,
 0.011342530607181281,
 0.23086752313855502,
 -0.81454587289996,
 -0.09618175525980585,
 -1.0319236066816422,
 -0.8941546899747144,
 -0.1769757567452964,
 -1.0261550115944087,
 -1.1428896872766254,
 0.5274096769373783,
 -0.07746401417773276,
 -0.6491216596122729,
 -0.08261022620000591,
 -1.057434038041655,
 -0.1560604072214138,
 -0.7524106597935429,
 -0.14270611717490567,
 0.3555326978742409,
 0.05380765583092719,
 -0.5460109939087803,
 0.5988517956598833,
 -0.20452965831003034,
 2.2859358114107895,
 -0.29751555947455255,
 -0.0045697871784828314,
 -0.0937335364724979,
 1.2297253567712267,
 -0.04174936326489904,
 -0.9955449547742778,
 1.0565721465309699,
 0.31961775158446143,
 0.15271746510289883,
 -1.1177224978944704,
 -0.05313924430296926,
 0.0964350617163593,
 -0.29751555947455255,
 0.012569615960219237,
 -0.009729483072938791,
 -0.5981403505268881,
 0.8665382507855257,
 0.7197226851876661,
 -0.6237064116985377,
 -1.01

In [79]:
n=0
for i in range(len(weight_list)):
    if (weight_list[i] > 0.15) and (1 == X_valid["Popularity"][i]):
        n = n+1
    elif (weight_list[i] <= 0.15) and (-1 == X_valid["Popularity"][i]):
        n = n+1
    #print(str(i) + ": \t" + str(X_valid["Popularity"][i]) +"\t"+str(weight_list[i]))
print("accu: " + str(n/len(weight_list)))

accu: 0.5304199921496795
